# Train DDQN for trailer reversing
Here we instantiate a simulation of the trailer system, and train a DDQN agent to control it.  

Imports

In [ ]:
# Import dependencies
import torch
import numpy as np
from numpy import array
import gym
from collections import namedtuple
from dqn_model import DoubleQLearningModel, ExperienceReplay
from IPython.core.debugger import set_trace
import dqn_model
from bicycleEnv import *

#Local files. 
from visualize_combination_code import *
from Simulate_combination_code import *
from utility_functions import *
from DDQNhelpers import * 

#dqn_model.test_calculate_q_targets(calculate_q_targets)

In [ ]:
# CPU should be enough, but feel free to play around with this if you want to.
device = torch.device("cpu")

## Actions
Define a set of actions $A = \left(a_1, a_2,..., a_n \right)$. 

Currently we have two potentinal control signals, velocity, 

$v \in (-1, 1)$

being forward and reverse, and steering angle $\delta$ which we do not know yet how to handle. For now we let 

$\delta \in \{-\delta_0, 0 , \delta_0 \}$

where $\delta_0 \in (-60,60)$



In [ ]:
#Actions are currently steering angle, where we either 
delta_0 = 45
actions = [-delta_0, 0,delta_0]

Instantiate a system model and run a simulation for a sanity check. 

In [ ]:
##System initial conditions. 


L = 2 #Length rear axis to front axis
Ts = 0.01 #Sample interval in seconds. 

#Position x, y, and heading
initState = (0,0, 0)

#Yes, the truck is a bicycle these days. 
truck = BicycleEnv(L,Ts, initState)


truck_pos_x = []
truck_pos_y = []
truck_angle = []

for step_number in range(np.int(1e3)):
    
        #velocity = -0.1+np.sin(step_number/1e3*np.pi)
        #steering_percentage = np.sin(step_number/1e2*np.pi)
        steeringAngle = np.deg2rad(-45)
        velocity  = 1
        
        action = (velocity, steeringAngle)
        state  = truck.step(action) 
        
        truck_pos_x.append(state[0])
        truck_pos_y.append(state[1])
        truck_angle.append(state[2])
        #truck_rot.append(truck_rotation)
        #trailer1_rot.append(first_trailer_rotation)
        #trailer2_rot.append(second_trailer_rotation)
        
#Reset before training on network.         
truck.reset()



Animate data for a sanity check

In [ ]:
from animateRectangle import* 

%matplotlib notebook 

#The previous code was put in a separate class, try it here. 
#%matplotlib tk 
fig= plt.figure()
#ax.axis('equal')
fig.set_dpi(100)
fig.set_size_inches(3, 3)
B =0

rectAnim = animateRectangle(fig, B,L, truck_pos_x, truck_pos_y, truck_angle)

anim = rectAnim.animate(Ts*1000)
plt.show()
plt.grid()

Train.

In [ ]:
# Create the environment
#env = gym.make("CartPole-v0")

# Enable visualization? Does not work in all environments.
enable_visualization = False

#Actions are full turn left, straight, full turn right
#actions = (-1,0,1)


# Initializations
num_actions = len(actions)
num_states = len(initState)

#Training hyperparameters. 
num_episodes = 50
batch_size = 128
gamma = .94
learning_rate = 1e-4

# Object holding our online / offline Q-Networks
ddqn = DoubleQLearningModel(device, num_states, num_actions, learning_rate)

# Create replay buffer, where experience in form of tuples <s,a,r,s',t>, gathered from the environment is stored 
# for training
replay_buffer = ExperienceReplay(device, num_states)

# Train
#set_trace()
#R, R_avg = train_loop_ddqn(ddqn, env, replay_buffer, num_episodes, enable_visualization=enable_visualization, batch_size=batch_size, gamma=gamma)

#DDQNhelpers.
#DDQNhelpers.
#set_trace()
R, R_avg = DDQNhelpers.train_loop_ddqn( truck, ddqn, replay_buffer, num_episodes, enable_visualization, batch_size, gamma)

In [ ]:
DDQNhelpers.train_loop_ddqn

In [ ]:
#for episode in train_log: 


#print(type(episode))
#plt.plot( episode[pos_x], episode[pos_y] )

def printEpisode(episode, i ):
    #Plot startpoint
    plt.plot(episode.pos_x[0],episode.pos_y[0],  'bo' )

    #Plot whole trajectory. 
    plt.plot( episode.pos_x, episode.pos_y, label = "Episode " + str(i) )
    #legend()
    plt.gca().legend()
    plt.plot(episode.pos_x[-1],episode.pos_y[-1],  'kx' )
    
    
    
for i, episode in enumerate(train_log):
    if (i%5==0):
        printEpisode(episode,i)
    
#
#episode_end = train_log[-1]
#episode_middle = train_log[4]
#episode_first = train_log[0]
#
#printEpisode(episode_end)
#printEpisode(episode_middle)
#printEpisode(episode_first)
#printEpisode(train_log[1])
#    
    
plt.figure() 

#for episode in train_log: 
#plt.plot(episode.steering_angle)
    